Tests COS performance

In [ ]:
!pip install ibm-cos-sdk python-dotenv pandas

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError
from dotenv import load_dotenv
import os

In [ ]:
%load_ext dotenv
%dotenv cos_performance_test.credentials
%dotenv cos_performance_test.env

aws_access_key_id=os.environ.get('aws_access_key_id')
aws_secret_access_key=os.environ.get('aws_secret_access_key')
cos_endpoint_url=os.environ.get('cos_endpoint_url')

# block size 4 MB is default (recommended)
block_size=int(os.environ.get('block_size','4194304'))
          
bucket=os.environ.get('bucket')
key=os.environ.get('key')
                


In [ ]:
cos_client = ibm_boto3.client("s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url=cos_endpoint_url
)

In [ ]:
"""
def get_all_s3_objects(s3, **base_kwargs):
    continuation_token = None
    while True:
        list_kwargs = dict(MaxKeys=1000, **base_kwargs)
        if continuation_token:
            list_kwargs['ContinuationToken'] = continuation_token
        response = s3.list_objects_v2(**list_kwargs)
        yield from response.get('Contents', [])
        if not response.get('IsTruncated'):  # At the end of the list?
            break
        continuation_token = response.get('NextContinuationToken')

for file in get_all_s3_objects(cos_client, Bucket='sentinel-1'):
    print(file)
"""

In [ ]:
import time
from random import randrange

In [ ]:
import pandas as pd


In [ ]:
def penetrate(blocks, bs) -> pd.DataFrame:
    df = pd.DataFrame(columns=['time1','time2'])
    
    for i in range(100):
        start_bytes = randrange(1019)*block_size
        stop_bytes = start_bytes+bs
        start = time.time()
        resp = cos_client.get_object(Bucket=bucket, Key=key, Range='bytes={}-{}'.format(start_bytes, stop_bytes))
        end = time.time()
        time1 = end - start
        start = time.time()
        res = resp['Body'].read()
        end = time.time()
        time2 = end - start
        print(f'{time1},{time2}')
        df.loc[len(df.index)] = [time1, time1]
    return df
        

In [ ]:
blocks = int(4277006357 / block_size) # TODO obtain object size from COS object
df1 = penetrate(blocks, block_size)

In [ ]:
df1['time1'].mean()+df1['time2'].mean()

In [ ]:
blocks = int(4277006357 / block_size-1000000) # TODO obtain object size from COS object
df2 = penetrate(blocks, block_size-1000000)

In [ ]:
df2['time1'].mean()+df2['time2'].mean()

In [ ]:
blocks = int(4277006357 / block_size+1000000) # TODO obtain object size from COS object
df3 = penetrate(blocks, block_size+1000000)

In [ ]:
df3['time1'].mean()+df3['time2'].mean()